In [1]:
import numpy as np

import pandas as pd

In [2]:
movie_info = "./ml-latest-small/movies.csv"

data = pd.read_csv(movie_info)

data[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
data = np.array(data)

count = 0

new_list = []

title_list = []

for row in data:
    title_info = row[1]
    genres_info = row[2]
    info_list = title_info.split('(')
    if len(info_list) == 2:
        title, year = info_list
        title = title.rstrip()
        year = year.rstrip()[:-1]
        
        if len(year) == 4:
            if int(year) >= 2010:
                new_list.append([count, title, year, genres_info])
                
                title_list.append(title)
                
                count += 1
                  
count 

1291

In [4]:
# with open("new_movieID.txt", "w", encoding="utf8") as f:
#     for i in new_list:
#         list_temp = [str(item) for item in i]
#         f.write(str(",".join(list_temp)))
#         f.write("\n")

In [5]:
new_list[:5]

[[0, 'Daybreakers', '2010', 'Action|Drama|Horror|Thriller'],
 [1, 'Leap Year', '2010', 'Comedy|Romance'],
 [2, 'Book of Eli, The', '2010', 'Action|Adventure|Drama'],
 [3, 'Legion', '2010', 'Action|Fantasy|Horror|Thriller'],
 [4, "Dante's Inferno Animated", '2010', 'Action|Animation|Fantasy']]

In [6]:
from TwitterAPI import TwitterAPI
import configparser
import sys
import time

In [7]:
# Get some tweets about McDonald's

def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = configparser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')

In [8]:
tweets = []

for r in twitter.request('search/tweets',
                         {'q': 'mcdonalds', 'count': 100}):
    tweets.append(r)

In [9]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request; e.g., "friends/ids"
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print('Got error %s \nsleeping for 15 minutes.' % request.text)
            sys.stderr.flush()
            time.sleep(61 * 15)

In [ ]:
result = {}

for title in title_list:
    
    new_title = title + " movie"
    query = {'q': new_title, 'count': 100}
    temp_result = robust_request(twitter, 'search/tweets', query, max_tries=5)
    
    if temp_result is not None:
        result[title] = temp_result

In [ ]:
print(len(result))

In [ ]:
new_result = {}

for key, value in result.items():
    count = 0
    value = list(value)
    new_result[key] = value

In [ ]:
import json
with open('result.json', 'w') as fp:
    json.dump(new_result, fp)